# Automatic refinement of elements

---

_Developed by D.A. Brakenhoff, December 2023, Artesia + TU Delft_


This notebook shows how elements can be automatically refined in TimML. 

## Contents

- [Supported elements](#supported-elements)
- [User added elements vs. refined elements](#user-added-elements-vs-refined-elements)
- [Single line-sink](#single-line-sink)
- [Compound line-sinks](#compound-line-sinks)
- [Compound line-sink with nearby well](#compound-line-sink-with-nearby-well)
- [Refining inhomogeneities: LeakyBuildingPit](#refining-inhomogeneities-leakybuildingpit)
- [Global refine option](#global-refine-option)

Refinement means splitting line elements into smaller sub-elements. This can be
necessary for computational accuracy at locations where elements lie close together.
The sub-elements derive their properties from the original user-specified elements.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import timml as tml
from timml.util import refine_n_segments

## Supported elements

The following elements support automatic refinement:

* Line-sinks and line-doublets:
    * LineSinkBase
    * HeadLineSink
    * HeadLineSinkString
    * LineSinkDitchString
    * ImpLineDoublet
    * ImpLineDoubletString
    * LeakyLineDoublet
    * LeakyLineDoubletString
* Inhomogeneities:
    * PolygonInhomMaq
    * PolygonInhom3D
    * BuildingPitMaq
    * BuildingPit3D
    * LeakyBuildingPitMaq
    * LeakyBuildingPit3D

## User added elements vs. refined elements

In order to support automatic refinement, TimML now distinguishes between elements
added to a model by the user, and elements used in the computation. When automatic
refinement is applied, in some cases, new elements are created and added to the
computation list. When an element is not refined, the original user specified element
is passed on to the computation list.

User-specified elements are stored under `ml.elements`:

In [ ]:
ml = tml.ModelMaq()
ls = tml.HeadLineSink(ml, refine_level=2)

ml.elements  # user-specified elements

The computation element list (`ml.elementlist`) is empty until the model is initialized
(or solved).

In [ ]:
ml.elementlist

Initialize the model

In [ ]:
ml.initialize()

Note that the computation list has been filled, in this case with 2 refined elements
based on the original HeadLineSink:

In [ ]:
ml.elementlist

## Single line-sink

Supported elements can be refined by passing the `refine_level` kwarg. A refinement
level of 0 or 1 means no refinement is applied. A refinement level of 3 means that a
line-sink is split into 3 segments. The segmentation is performed according to an
adjusted version of the cosine rule (the same method that determines the location of
the control points for a line element in TimML).

In this example a single `HeadLineSink` is refined into 3 segments. The head along the
line-sink is compared to a case with no refinement.

First we define some model parameters. We have a single semi-confined aquifer split
into two 10 m-thick layers, with 1 day resistance between the two layers. The confining
layer has resistance of 1000 days with head of 0 m+ref above the confining layer.

In [ ]:
# model parameters

kh = 10  # m/day

ctop = 1000.0  # resistance top leaky layer in days

ztop = 0.0  # surface elevation
zbot = -20.0  # bottom elevation of the model
z = np.array([ztop + 1, ztop, -10, -10, zbot])

Build and store the models. Note that the only change required to apply the automatic
refinement is to supply a `refine_level>1` to the `HeadLineSink` element.

In [ ]:
models = []

for rlvl in [1, 3]:
    ml = tml.ModelMaq(kaq=kh, z=z, c=[ctop, 1], topboundary="semi", hstar=0.0)
    ls = tml.HeadLineSink(ml, 0, 0, 10, 10, hls=1.0, refine_level=rlvl)
    ml.solve(silent=True)
    models.append(ml)

Plot a top-view of the model.

In [ ]:
models[0].plots.topview([-10, 20, -10, 20])
plt.xlabel("x [m]")
plt.ylabel("y [m]");

Compare the head along the element for the refined and non-refined models. The refined
element has three points at which the head condition is specified, whereas the initial
element of order 0 has just 1 control point.

In [ ]:
# compute distance along line-sink
xy = np.array([(0, 0), (10, 10)])
x, y = refine_n_segments(xy, "line", 101)[0].T
r = np.sqrt((x - x[0]) ** 2 + (y - y[0]) ** 2)

# loop over models
for i, iml in enumerate(models):
    h = iml.headalongline(x, y)
    if i == 0:
        lbl = "No refinement"
        # plot head condition
        plt.plot([0, r[-1]], [ls.hls, ls.hls], ls="dashed", color="k")
    else:
        lbl = f"Refinement level = {ls.refine_level}"

    # plot head
    plt.plot(r, h[0], c=f"C{i}", label=lbl)

    # plot locations control points
    for e in iml.elementlist:
        if isinstance(e, tml.HeadLineSink):
            rc = np.sqrt((e.xc - x[0]) ** 2 + (e.yc - y[0]) ** 2)
            if i == 0:
                plt.plot(rc, ls.hls, f"C{i}o", ms=5, zorder=5)
            else:
                plt.plot(rc, ls.hls, f"C{i}o", ms=10, mfc="none", zorder=5)
leg = plt.legend(loc=(0, 1), frameon=False, ncol=2)
plt.grid(True)

<div class="alert alert-block alert-warning"><b>Note: </b> 
Be careful when refining single line-sinks! The variable referring to the original
element is not the one used in the calculations! See below for more information. 
</div>

When refining a single line-sink, the TimML creates new refined elements internally. This means that the original user-specified line-sink was not used in the computation and cannot be used for calculations, e.g. getting the discharge of the line-sink.


This means the following will not work:

In [ ]:
ls = models[1].elements[-1]  # get user-specified element from refined model


# NOTE: since the original element is never used in computation, it is never
# initialized and will throw an error when trying to do computations with it.
try:
    ls.discharge()
except Exception as e:
    print(e.__class__.__name__, e)

Instead do this:

In [ ]:
# the refined HeadLineSink elements are the last three in the elementlist
models[1].elementlist

In [ ]:
Q = np.zeros(models[1].aq.naq)  # 2 layers
for e in models[1].elementlist[1:]:  # loop through computation (refined) elements
    Q += e.discharge()
Q

Another (better) method to avoid this issue is to use compound line-sink elements, such as
`HeadLineSinkString`, which is shown in the next section.

## Compound line-sinks

Compound line-sinks are elements that consist of multiple line-sinks, e.g.
`HeadLineSinkString`. Refining these elements works similar to the example for a single
line-sink. 

<div class="alert alert-block alert-info"><b>Tip: </b> 
The advantage of compound elements is that they store their own list of
sub-elements internally, which means the original element can be used for further
computation, unlike the example with a single line-sink.
</div>


In this example we have the same single line-sink as the previous example, but the
specified head is 1 m+ref at the starting point and 0 m+ref at the end of the line-sink.
Because the head is sloping we need more than 1 control point in the non-refined model,
so we increase the order of the element to 2. That model is compared to a refined model
with a refine_level of 3 and order 0.

Build the models

In [ ]:
models = []

for rlvl, order in zip([1, 3], [2, 0], strict=False):
    ml = tml.ModelMaq(kaq=kh, z=z, c=[ctop, 1], topboundary="semi", hstar=0.0)
    ls = tml.HeadLineSinkString(
        ml, xy=[(0, 0), (10, 10)], hls=[1.0, 0.0], refine_level=rlvl, order=order
    )
    ml.solve(silent=True)
    models.append(ml)

Compare the head contours between the two models. As expected they look quite similar.

In [ ]:
models[-1].plots.topview([-20, 30, -20, 30])
for i, iml in enumerate(models):
    iml.plots.contour([-20, 30, -20, 30], 101, newfig=False, decimals=2, color=f"C{i}")

As mentioned previously, the advantage of compound line-sinks is that the original
reference to the line-sink that was specified by the user can be used for computation.

In [ ]:
# the HeadLineSinkString is the second element in ml.elements
models[0].elements

In [ ]:
# non-refined model
models[0].elements[1].discharge()

In [ ]:
# refined model
models[1].elements[1].discharge()

## Compound line-sink with nearby well

In this example a well is pumping near a head-specified line-sink. We apply different
refinement levels and observe the effect on the total discharge of the line-sink. 

First specify the line-sink and well coordinates.

In [ ]:
xy = np.array([(0, 0), (10, 10), (20, 10)])  # line-sink coordinates

xw, yw = 15, 0  # well coordinates
rw = 0.3  # well radius, in m
Qw = 100.0  # well discharge, in m3/d

Plot a top-view of the model

In [ ]:
plt.plot(xy[:, 0], xy[:, 1], "k-o", label="head-specified line-sink")
plt.plot(xw, yw, "C0o", label="well")
leg = plt.legend(loc=(0, 1), frameon=False, ncol=2)
plt.xlabel("x [m]")
plt.ylabel("y [m]");

Build models and apply refinement levels 1-9. Print the total discharge of the line-sink for each model. After level 4, the discharge does not change by all that much any more. 

In [ ]:
models = []
for rlvl in range(1, 10):
    ml = tml.ModelMaq(kaq=kh, z=z, c=[ctop, 1], topboundary="semi", hstar=0.0)
    hls = tml.HeadLineSinkString(ml, xy, hls=[2, 1], refine_level=rlvl)
    w = tml.Well(ml, xw, yw, Qw, rw=0.1, layers=[0])
    ml.solve(silent=True)
    models.append(ml)
    print(f"Refinement level {rlvl}: Qls = {hls.discharge().sum():.2f}")

Compare the head contours for the first (non-refined) and last model (`refine_level=9`)

In [ ]:
models[0].plots.topview([-10, 30, -10, 30])
models[0].plots.contour([-10, 30, -10, 30], 101, newfig=False, decimals=2, color="C0")
models[-1].plots.contour([-10, 30, -10, 30], 101, newfig=False, decimals=2, color="C1")

plt.xlabel("x [m]")
plt.ylabel("y [m]");

Compare the head along the line-sink to the specified head-conditions for each model.

In [ ]:
for i, iml in enumerate(models):
    x, y = refine_n_segments(xy, "line", 101)[0].T
    r = np.sqrt((x - x[0]) ** 2 + (y - y[0]) ** 2)
    h = iml.headalongline(x, y)
    plt.plot(r, h[0], c=f"C{i}", label=f"{i + 1}")

plt.plot(r[[0, -1]], hls._hls[[0, -1]], ls="dashed", color="k")
plt.xlabel("distance along line-sink [m]")
plt.ylabel("head [m+ref]")
plt.legend(loc=(0, 1), frameon=False, ncol=9, fontsize="x-small")
plt.grid(True)

## Refining inhomogeneities: LeakyBuildingPit

In this example a model with an inhomogeneity is refined. In this case we're refining a
rectangular LeakyBuildingPit with a sheetpile wall that has an effective resistance of
100 days on three sides. On the northern side, the sheetpile wall has almost no
resistance. The bottom of the sheetpile wall reaches halfway into the aquifer.

The model is confined, and a well is pumping inside the leaky building pit with 100
$m^3$/day. Define the coordinates of the leaky building pit and the resistance of the
walls.

In [ ]:
xy = [
    (-10, -5),
    (10, -5),
    (10, 5),
    (-10, 5),
    (-10, -5),
]

res = np.array([100.0, 100.0, 1e-3, 100.0])  # resistance of leaky wall, in days

Build the model, without refinement.

In [ ]:
ml = tml.ModelMaq(kaq=kh, z=z, c=[ctop, 1], topboundary="semi", hstar=0.0)
bpit = tml.LeakyBuildingPitMaq(
    ml,
    xy,
    kaq=kh,
    z=z[1:],
    topboundary="conf",
    c=[1],
    layers=[0],
    res=res,
)
well = tml.Well(ml, 0.0, 0.0, Qw=Qw, rw=rw)
ml.solve()

Plot the head contours in both layers. Note the head contours in the corners of the
leaky building pit. Clearly, the solution isn't quite right at these locations.

In [ ]:
tiny = 1e-5
xgr = np.linspace(-10 + tiny, 10 - tiny, 101)
ygr = np.linspace(-5 + tiny, 5 - tiny, 51)
h = ml.headgrid(xgr, ygr)
plt.contour(xgr, ygr, h[0], levels=20, colors="C0")
plt.contour(xgr, ygr, h[1], levels=20, colors="C1")
plt.axis("scaled")
plt.xlabel("x [m]")
plt.xlabel("y [m]");

We can also see that the water balance isn't quite correct. In the first layer, along 3
sides, there is flow out of the building pit (negative numbers), which is not
what we would expect, and the total discharge flowing into the building pit should
equal the pumping discharge of the well (it's close but not quite right).

In [ ]:
df = pd.DataFrame(
    index=np.arange(ml.aq.naq),
    columns=["S", "E", "N", "W"],
    data=ml.intnormflux(xy, ndeg=99),
)
df.index.name = "Layer"
df.columns.name = "Side"
df["total"] = df.sum(axis=1)
df.loc["total", :] = df.sum(axis=0)
df.round(1)

Next, let's refine the leaky building pit, and see how that affects the solution. Let's
try a `refine_level` of 3.

In [ ]:
mlr = tml.ModelMaq(kaq=kh, z=z, c=[ctop, 1], topboundary="semi", hstar=0.0)
bpitr = tml.LeakyBuildingPitMaq(
    mlr,
    xy,
    kaq=kh,
    z=z[1:],
    topboundary="conf",
    c=[1],
    layers=[0],
    res=res,
    refine_level=3,
)
wellr = tml.Well(mlr, 0.0, 0.0, Qw=Qw, rw=rw)
mlr.solve()

Plot the contours. The results in the corners of the building pit seem a lot more
realistic, though there are still some visible minor irregularities in the top-left and
right corners.

In [ ]:
tiny = 1e-5
xgr = np.linspace(-10 + tiny, 10 - tiny, 101)
ygr = np.linspace(-5 + tiny, 5 - tiny, 51)
h = mlr.headgrid(xgr, ygr)
plt.contour(xgr, ygr, h[0], levels=20, colors="C0")
plt.contour(xgr, ygr, h[1], levels=20, colors="C1")
plt.axis("scaled")
plt.xlabel("x [m]")
plt.xlabel("y [m]");

The calculated discharge is now more realistic, with no discharge out of the building
pit in layer 0 and the total discharge is exactly equal to the discharge of the pumping
well.

In [ ]:
df = pd.DataFrame(
    index=np.arange(mlr.aq.naq),
    columns=["S", "E", "N", "W"],
    data=mlr.intnormflux(xy, ndeg=99),
)
df.index.name = "Layer"
df.columns.name = "Side"
df["total"] = df.sum(axis=1)
df.loc["total", :] = df.sum(axis=0)
df.round(1)

## Global refine option

In the examples above the refine_level was defined in the elements that were meant to
be refined. This allows for fine-grained control over which elements should be refined
and by how much. This is the preferred method for specifying this information. However,
in certain situations it can be useful to globally set a refinement level.

This is possible by setting the `refine_level` in `ml.solve()`. Setting this keyword
argument to None (the default), uses the element-level settings, setting it to a number
will override the element settings.

In [ ]:
ml = tml.ModelMaq(kaq=kh, z=z, c=[ctop, 1], topboundary="semi", hstar=0.0)
ls = tml.HeadLineSink(ml, 0, 0, 10, 10, hls=1.0)  # no refine_level specified
ml.solve(refine_level=3)

In [ ]:
ml.elementlist